### TCLab Stability Analysis

See [TCLab Stability Analysis](https://apmonitor.com/pdc/index.php/Main/TCLabStabilityAnalysis) for additional information, including solution help.

**Overview**: Stability analysis for the Temperature Control Lab is finding the range of controller gains that lead to a stabilizing controller. There are multiple methods to compute this range.

**Objective**: Determine closed-loop behavior of a P-only controller with the TCLab including oscillatory behavior and stability. [Background information on stability analysis](https://apmonitor.com/pdc/index.php/Main/StabilityAnalysis) may be useful for completing this exercise.

In [ ]:
pip install gekko

For the temperature control lab, determine a second-order model between heater 1 ($Q_1$) and temperature sensor 1 ($T_{C1}$).

$G_p(s) = \frac{T_{C1}(s)}{Q_1(s)} = \frac{K_p}{\tau_s^2\,s^2+2\,\zeta\,\tau_s\,s+1}$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from gekko import GEKKO
import tclab
import time

# Import data
try:
    # try to read local data file first
    filename = 'data.csv'
    data = pd.read_csv(filename)
except:
    # heater steps
    Q1d = np.zeros(601)
    Q1d[10:200] = 80
    Q1d[200:280] = 20
    Q1d[280:400] = 70
    Q1d[400:] = 50

    Q2d = np.zeros(601)

    try:
        # Connect to Arduino
        a = tclab.TCLab()
        fid = open(filename,'w')
        fid.write('Time,Q1,Q2,T1,T2\n')
        fid.close()

        # run step test (10 min)
        for i in range(601):
            # set heater values
            a.Q1(Q1d[i])
            a.Q2(Q2d[i])
            print('Time: ' + str(i) + \
                  ' Q1: ' + str(Q1d[i]) + \
                  ' Q2: ' + str(Q2d[i]) + \
                  ' T1: ' + str(a.T1)   + \
                  ' T2: ' + str(a.T2))
            # wait 1 second
            time.sleep(1)
            fid = open(filename,'a')
            fid.write(str(i)+','+str(Q1d[i])+','+str(Q2d[i])+',' \
                      +str(a.T1)+','+str(a.T2)+'\n')
            fid.close()
        # close connection to Arduino
        a.close()
    except:
        filename = 'https://apmonitor.com/pdc/uploads/Main/tclab_data3.txt'
    # read either local file or use link if no TCLab
    data = pd.read_csv(filename)

# Second order model of TCLab
m = GEKKO()
m.time = data['Time'].values
Kp   = m.FV(1.0,lb=0.5,ub=2.0)
taus = m.FV(50,lb=10,ub=200)
zeta =  m.FV(1.2,lb=1.1,ub=5)
T0 = data['T1'][0]
Q1 = m.Param(0)
x = m.Var(0); TC1 = m.CV(T0)
m.Equation(x==TC1.dt())
m.Equation((taus**2)*x.dt()+2*zeta*taus*TC1.dt()+(TC1-T0) == Kp*Q1)
m.options.IMODE = 5
m.options.NODES = 2
m.options.EV_TYPE = 2 # Objective type
Kp.STATUS = 1
taus.STATUS = 1
zeta.STATUS = 1
TC1.FSTATUS = 1
Q1.value=data['Q1'].values
TC1.value=data['T1'].values
m.solve(disp=False)

# Parameter values
print('Estimated Parameters')
print('Kp  : ' + str(Kp.value[0]))
print('taus: ' + str(taus.value[0]))
print('zeta: ' + str(zeta.value[0]))

# Create plot
plt.figure(figsize=(10,7))
ax=plt.subplot(2,1,1)
ax.grid()
plt.plot(data['Time'],data['T1'],'b.',label=r'$T_1$ measured')
plt.plot(m.time,TC1.value,color='orange',linestyle='--',\
         linewidth=2,label=r'$T_1$ second order')
plt.ylabel(r'T ($^oC$)')
plt.legend(loc=2)
ax=plt.subplot(2,1,2)
ax.grid()
plt.plot(data['Time'],data['Q1'],'r-',\
         linewidth=3,label=r'$Q_1$')
plt.ylabel('Heater (%)')
plt.xlabel('Time (sec)')
plt.legend(loc='best')
plt.savefig('tclab_2nd_order.png')
plt.show()

With the second-order TCLab model, determine the range of controller gains ($K_c$) where a P-only controller oscillates and is stable (does not diverge). Traditional stability analysis does not include information about controller output saturation (0-100% heater).

#### P-Only Simulator

Use the P-Only simulator to test the stability and oscillation predictions. The simulator shows the stability and oscillation of a P-only controller with a TCLab second-order model with $K_p$=0.8473 $^o$C, $\tau_s$=51.08 sec, $\zeta$=1.581 sec, and $\theta_p$=0.0 sec. Use the second-order model parameters from your own TCLab device for a more accurate simulation. The controller gain $K_c$ is adjusted with a slider to compute the updated temperature response.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as wg
from IPython.display import display

n = 601 # time points to plot
tf = 600.0 # final time

# TCLab Second-Order
Kp = 0.8473
taus = 51.08
zeta = 1.581
thetap = 0.0

def process(z,t,u):
    x,y = z
    dxdt = (1.0/(taus**2)) * (-2.0*zeta*taus*x-(y-23.0) + Kp * u)
    dydt = x
    return [dxdt,dydt]

def pidPlot(Kc):
    t = np.linspace(0,tf,n) # create time vector
    P = np.zeros(n)         # initialize proportional term
    e = np.zeros(n)         # initialize error
    OP = np.zeros(n)        # initialize controller output
    PV = np.ones(n)*23.0    # initialize process variable
    SP = np.ones(n)*23.0    # initialize setpoint
    SP[10:] = 60.0          # step up
    z0 = [0,23.0]           # initial condition
    # loop through all time steps
    for i in range(1,n):
        # simulate process for one time step
        ts = [t[i-1],t[i]]         # time interval
        z = odeint(process,z0,ts,args=(OP[max(0,i-1-int(thetap))],))
        z0 = z[1]                  # record new initial condition
        # calculate new OP with PID
        PV[i] = z0[1]              # record PV
        e[i] = SP[i] - PV[i]       # calculate error = SP - PV
        dt = t[i] - t[i-1]         # calculate time step
        P[i] = Kc * e[i]           # calculate proportional term
        OP[i] = min(100,max(0,P[i])) # calculate new controller output        

    P = np.zeros(n)         # initialize proportional term
    e = np.zeros(n)         # initialize error
    OPu = np.zeros(n)       # initialize controller output
    PVu = np.ones(n)*23.0   # initialize process variable
    SP = np.ones(n)*23.0    # initialize setpoint
    SP[10:] = 60.0          # step up
    z0 = [0,23.0]           # initial condition
    # loop through all time steps
    for i in range(1,n):
        # simulate process for one time step
        ts = [t[i-1],t[i]]         # time interval
        z = odeint(process,z0,ts,args=(OPu[max(0,i-1-int(thetap))],))
        z0 = z[1]                  # record new initial condition
        # calculate new OP with PID
        PVu[i] = z0[1]             # record PV
        e[i] = SP[i] - PVu[i]       # calculate error = SP - PV
        dt = t[i] - t[i-1]         # calculate time step
        P[i] = Kc * e[i]           # calculate proportional term
        OPu[i] = P[i]               # calculate new controller output
        
    # plot PID response
    plt.figure(1,figsize=(15,5))
    plt.subplot(1,2,1)
    plt.plot(t,SP,'k-',linewidth=2,label='Setpoint (SP)')
    plt.plot(t,PV,'r-',linewidth=2,label='Temperature - OP Limits (PV)')
    plt.plot(t,PVu,'b--',linewidth=2,label='Temperature - No OP Limits (PV)')
    plt.ylabel(r'T $(^oC)$')
    plt.text(100,30,'OP Limit Offset: ' + str(np.round(SP[-1]-PVu[-1],2)))
    M = SP[-1]-SP[0]
    pred_offset = M*(1-Kp*Kc/(1+Kp*Kc))
    plt.text(100,25,'No OP Limit Offset: ' + str(np.round(pred_offset,2)))
    plt.text(400,30,r'$K_c$: ' + str(np.round(Kc,1)))  
    plt.legend(loc=1)
    plt.xlabel('time (sec)')
    plt.subplot(1,2,2)
    plt.plot(t,OP,'r-',linewidth=2,label='Heater - OP Limits (OP)')
    plt.plot(t,OPu,'b--',linewidth=2,label='Heater - No OP Limits (OP)')
    plt.ylabel('Heater (%)')
    plt.legend(loc='best')
    plt.xlabel('time (sec)')

Kc_slide = wg.FloatSlider(value=2.0,min=-2.0,max=400.0,step=1.0)
wg.interact(pidPlot, Kc=Kc_slide)
print('P-only Simulator with and without OP Limits: Adjust Kc')

See [Solution Section](https://apmonitor.com/pdc/index.php/Main/TCLabStabilityAnalysis) and [Video](https://youtu.be/CRthog3EsbM) for additional help.